In [1]:
import pandas as pd
import numpy as np
import os
import random
import sys
import glob 
import librosa
# import librosa.display
import matplotlib.pyplot as plt
# import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
# classification_report, confusion_matrix, 
# from sklearn.preprocessing import StandardScaler
# from sklearn.utils.multiclass import unique_labels
from sklearn.neural_network import MLPClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# import lightgbm as lgb
# import xgboost as xgb
# import optuna
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

print('DONE')

DONE


In [2]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft=np.abs(librosa.stft(X))
    result=np.array([])
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    result=np.hstack((result, mfccs))
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    result=np.hstack((result, chroma))
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    result=np.hstack((result, mel))
    return result

print('DONE')

DONE


In [3]:
emotions = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'afraid',
  '07':'disgusted',
  '08':'surprised'
}

def gender(g):
    if int(g[0:2]) % 2 == 0:
        return 'female'
    else:
        return 'male'

print('DONE')

DONE


In [4]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in tqdm(glob.glob("datasets_combined/Actor_*/*.wav")):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]] + '_' + gender(file_name.split("-")[-1])
        feature=extract_feature(file)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

print('DONE')

DONE


In [ ]:
X_train, X_test, y_train, y_test = load_data(0.2)

 87%|████████▋ | 2121/2452 [06:29<01:04,  5.09it/s]

In [23]:
print((X_train.shape[0], X_test.shape[0]))

(1839, 613)


In [24]:
print(f'Features extracted: {X_train.shape[1]}')

Features extracted: 180


In [25]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print('DONE')

DONE


In [26]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 56.61%


In [10]:
import pickle
with open('final64.pkl', 'wb') as file:
    pickle.dump(model, file)